# Detecting coordinates

To create masks regionmask needs to know the x and y (longitude and latitude) coordinates of the grid. There are two ways to pass these to the `mask` and `mask_3D` methods:

1. Passing the coordinates explicitly: `region.mask(longitude, latitude)`
2. Pass an xarray `DataArray` or `Dataset`: `region.mask(ds)` in which case the coordinates are either auto-detected using [cf_xarray](https://cf-xarray.readthedocs.io/en/latest/coord_axes.html) or determined as `ds["lon"]` and `ds["lat"]`

## Illustration

### Preparation

Import regionmask and check the version:

In [ ]:
import regionmask

print(regionmask.__version__)

Other imports

In [ ]:
import numpy as np
import xarray as xr

xr.set_options(display_width=60, display_expand_data=False)

Get data for examples



In [ ]:
# example region
region = regionmask.defined_regions.ar6.land

# dataset with coordinates named "lon" and "lat", without cf metadata
ds = xr.Dataset(coords={"lon": np.arange(0, 360), "lat": np.arange(90, 0, -1)})

# dataset with coordinates not named "lon" and "lat," with cf metadata
rasm = xr.tutorial.open_dataset("rasm")

### 1. Passing coordinates directly

The coordinates can be passed individually - these must be "array-like" (typically numpy arrays or xarray DataArrays):

In [ ]:
mask = region.mask(ds.lon, ds.lat)
mask.plot()

### 2. Not using cf_xarray

If cf_xarray is not installed, the passed dataset does not have any cf metadata/ attributes, or `use_cf=False` is set, regionmask tries to read the coordinates from `ds["lon"]` and `ds["lat"]` and raises an error otherwise:

In [ ]:
mask = region.mask(ds)
mask

In [ ]:
try:
    region.mask(rasm, use_cf=False)
except KeyError:
    print("Error raised")

### Using cf_xarray

[cf_xarray](https://cf-xarray.readthedocs.io) uses [cf metadata](http://cfconventions.org/) saved in the attributes to determine coordinates (see [coordinate-criteria](https://cf-xarray.readthedocs.io/en/latest/coord_axes.html#coordinate-criteria)). For this the cf_xarray needs to be installed but importing it is not necessary.

In [ ]:
mask = region.mask_3D(rasm)

mask

## Details

regionmask uses the following logic to determine the coordinates:

1. Passing the coordinates directly (`region.mask(lon, lat)`) takes precedence.
1. If cf_xarray is _not_ installed regionmask will use `ds["lon"]` and `ds["lat"]` or raise a `KeyError`.
1. If cf_xarray is installed the behavior is determined via the `use_cf` keyword of the `mask` and `mask_3D` methods:
   * `use_cf=None` (default): use the cf metadata to determine the coordinates _or_ use `ds["lon"]` and `ds["lat"]` (raises a `ValueError` if ambigous).
   * `use_cf=True` : use the cf metadata to determine the coordinates
   * `use_cf=False`:  use `ds["lon"]` and `ds["lat"]` as coordinates